In [1]:
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

root = Path(os.getcwd()).parent.parent

#Change for each one of where your data is. For me in Dataset I have all the three folders
parent_directory = os.path.dirname(root)
dataset_path = os.path.join(root,'Dataset')

ADVOCATE = "advocate"
RATEBEER = "ratebeer"
MATCHED = "matched"
FULL = "full"

ADVOCATE_PATH = os.path.join(dataset_path,ADVOCATE)
RATEBEER_PATH = os.path.join(dataset_path,RATEBEER)
MATCHED_PATH = os.path.join(dataset_path,MATCHED)
FULL_PATH = os.path.join(dataset_path,FULL)

**Introduction**

The goal of this notebook is to found a way to transform every family dataset into one single dataset. At the end we should have one single user dataset, one single beer dataset and so on. Those datasets would be downloadable, except ratings where we want to create a function that we can run at the beginning of each script, this is done to avoid to have to download/load a much heavier dataset.

To facilitate our life we want to create a single id that could correspond accross the different datasets. It is important that there remains a way to connect to the dataset of origin, i.e that we do not drop the general id.

In [26]:
#Open this one already as it may become usefull for reweries already
beers_matched = pd.read_csv(os.path.join(MATCHED_PATH,'beers.csv'),header=1)

**Breweries dataset**

We saw that in data_understanding.ipynb, for breweries there were duplicates. So we need to treat them carefully.

In [27]:
breweries_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'breweries.csv'))
breweries_matched = pd.read_csv(os.path.join(MATCHED_PATH,'breweries.csv'),header=1)
breweries_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'breweries.csv'))

print("Length of the three datasets:\n-advocate:",len(breweries_advocate),"\n-matched:", len(breweries_matched),"\n-ratebeer:", len(breweries_ratebeer))

#Already delete the breweries presents in matched
breweries_ratebeer_solo = breweries_ratebeer[~breweries_ratebeer.id.isin(breweries_matched['id.1'])]
breweries_advocate_solo = breweries_advocate[~breweries_advocate.id.isin(breweries_matched['id'])]

Length of the three datasets:
-advocate: 16758 
-matched: 8281 
-ratebeer: 24189


Both advocate and ratebeer dataset have as columns ['id', 'location', 'name', 'nbr_beers']. Matched has those columns with .1 referring to ratebeer, they also have a diff and sim column. The end format we want to have: [general_id, old_id_advocate, 'old_id_ratebeer', location, name and nbr_beers]. However we need to be carefull to the duplicates of beers in the beers dataset. As the duplicates where for advocate and not Ratebeer we make the convention to take the name and location from Ratebeer. We also want to drop diff and sim.  
Let us first get the duplicates of breweries.

In [28]:
breweries_duplicates_ratebeer = breweries_matched[breweries_matched['id.1'].duplicated(keep=False)]
breweries__not_duplicated_ratebeer = breweries_matched[~breweries_matched['id.1'].duplicated(keep=False)]#For later
breweries_duplicates_advocate = breweries_matched[breweries_matched['id'].duplicated(keep=False)]

print("Number of duplicated/tripled Ratbeer breweries",len(breweries_duplicates_ratebeer))
print("Number of duplicated/tripled Advocate breweries",len(breweries_duplicates_advocate))

Number of duplicated/tripled Ratbeer breweries 91
Number of duplicated/tripled Advocate breweries 0


In [29]:
#We are going to build a new dataset instead of playing with the dataset that contains every information for duplicates.

unique_ratebeer_brewery_id = breweries_duplicates_ratebeer['id.1'].unique()
new_matched_brewery_duplicate = pd.DataFrame(columns=['location', 'name', 'nbr_beers','old_advocate_id','old_ratebeer_id'])

for id in unique_ratebeer_brewery_id:  # Replace with your actual loop condition
    # Define data for each row
    location = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['location.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    name = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['name.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    old_advocate_id = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['id']
    list_old_advocate_id = []
    for id_advocate in (old_advocate_id):
        list_old_advocate_id.append(id_advocate)
    old_ratebeer_id = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['id.1'].iloc[0]

    total_beers_advocate = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['nbr_beers'].sum()
    total_beers_ratebeer = breweries_duplicates_ratebeer[breweries_duplicates_ratebeer['id.1']==id]['nbr_beers.1'].iloc[0] #don't want to double the number of beers for the one duplicated

    nbr_beers = total_beers_advocate+ total_beers_ratebeer

    # Append the new row to the DataFrame
    new_row = pd.DataFrame({'location': location, 'name': name, 'nbr_beers': nbr_beers,'old_advocate_id':[list_old_advocate_id],'old_ratebeer_id':old_ratebeer_id})
    new_matched_brewery_duplicate = pd.concat([new_matched_brewery_duplicate, new_row], ignore_index=True)

new_matched_brewery_duplicate.sample(10)


,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
26,"United States, Virginia",Crooked Run Brewing,141,"[32661, 48463]",17177
0,England,Seven Bro7hers,11,"[37180, 45243]",20891
12,Australia,Cavalier Beer,98,"[25867, 36326]",12657
5,Canada,Skeena Brewing Company,2,"[38530, 13832]",11649
24,"United States, Maryland",DuClaw Brewing Company,307,"[1924, 16345]",779
17,"United States, Louisiana",Rikenjaks Brewing Company,6,"[44929, 970]",3860
15,Sweden,Carlsberg Sverige,265,"[5368, 10897]",765
37,"United States, Oregon",Pelican Pub & Brewery,202,"[48599, 1304]",1511
33,"United States, Ohio",Hoof Hearted Brewing,199,"[30179, 44305]",14572
29,"United States, North Carolina",Lynnwood Grill & Brewing Concern,60,"[47982, 33492]",19642


Did the next cell in a dumb way just need to rename the columns, would be faster.

In [30]:
unique_ratebeer_brewery_id = breweries__not_duplicated_ratebeer['id.1'].unique()
new_matched_brewery_non_duplicate = pd.DataFrame(columns=['location', 'name', 'nbr_beers','old_advocate_id','old_ratebeer_id'])

for id in unique_ratebeer_brewery_id:  # Replace with your actual loop condition
    # Define data for each row
    location = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['location.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    name = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['name.1'].iloc[0]#Always take the first as it is the same for the two/three of them
    old_advocate_id = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['id'].iloc[0]
    old_ratebeer_id = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['id.1'].iloc[0]

    total_beers_advocate = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['nbr_beers'].sum()
    total_beers_ratebeer = breweries__not_duplicated_ratebeer[breweries__not_duplicated_ratebeer['id.1']==id]['nbr_beers.1'].sum()

    nbr_beers = total_beers_advocate+ total_beers_ratebeer

    # Append the new row to the DataFrame
    new_row = pd.DataFrame({'location': location, 'name': name, 'nbr_beers': nbr_beers,'old_advocate_id':[old_advocate_id],'old_ratebeer_id':old_ratebeer_id})
    new_matched_brewery_non_duplicate = pd.concat([new_matched_brewery_non_duplicate, new_row], ignore_index=True)

new_matched_brewery_non_duplicate.sample(3)

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
3058,Czech Republic,Pivovar Chomout,41,39847,21344
6805,"United States, Oregon",Agrarian Ales,183,31704,15788
1352,Italy,Il Quarto dOra Granata (Alvemar srl),27,38859,16464


In [31]:
#Concat the two datasets together
new_matched_brewery = pd.concat([new_matched_brewery_non_duplicate, new_matched_brewery_duplicate], ignore_index=True)
new_matched_brewery.sample(3)

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
910,Canada,Hell Or High Water Beer And Spirits Inc,2,39897,22602
2643,Spain,Monsieur Gordo Brewery,8,46050,21308
954,Canada,Phillips Brewing Co.,309,2675,1844


In [32]:
print(len(new_matched_brewery), len(new_matched_brewery_non_duplicate), len(new_matched_brewery_duplicate))
print(len(new_matched_brewery.old_ratebeer_id.unique()))

8235 8190 45
8235


Now we need to count the number of beers that there really is. We assume that a beer can only be matched if its brewery is matched too.

In [33]:
new_matched_brewery_good_amount_rating = new_matched_brewery.copy()
for id in beers_matched['brewery_id.1']:
    new_matched_brewery_good_amount_rating.loc[new_matched_brewery_good_amount_rating['old_ratebeer_id'] == id, 'nbr_beers'] -= 1
    


In [34]:
new_matched_brewery_good_amount_rating.sample(3)

,location,name,nbr_beers,old_advocate_id,old_ratebeer_id
4066,"United States, California",Indian Joe Brewing,56,30546,15479
2146,Germany,Privatbrauerei Rogg,38,6790,9921
979,Canada,BRB &#40;Be Right Back - Big River&#41;,102,7527,3288


Now let's match the three datasets together.

In [35]:
breweries_ratebeer_solo = breweries_ratebeer_solo.rename(columns={'id': 'old_ratebeer_id'})
breweries_ratebeer_solo['old_advocate_id'] = np.nan
breweries_ratebeer_solo.sample(3)  

,old_ratebeer_id,location,name,nbr_beers,old_advocate_id
14465,5945,"United States, Washington",Elkhead Brewing Company,26,NaN
10000,26170,Vietnam,Tê Tê Brewing Co.,1,NaN
9296,28846,Austria,GROK,4,NaN


In [36]:
breweries_advocate_solo = breweries_advocate_solo.rename(columns={'id': 'old_advocate_id'})
breweries_advocate_solo['old_ratebeer_id'] = np.nan
breweries_advocate_solo.sample(3)  

,old_advocate_id,location,name,nbr_beers,old_ratebeer_id
16189,1887,"United States, Pennsylvania",Jack's Mountain Restaurant and Brewery,6,NaN
6580,34185,Brazil,Serra de Três Pontas Cervejaria Artesanal,4,NaN
635,71,England,Fuller Smith & Turner PLC,61,NaN


In [37]:
full_breweries = pd.concat([new_matched_brewery_good_amount_rating,breweries_advocate_solo, breweries_ratebeer_solo], ignore_index=True)
full_breweries['id'] = range(1, len(full_breweries) + 1)
full_breweries.sample(10)


,location,name,nbr_beers,old_advocate_id,old_ratebeer_id,id
22424,Australia,Currumbin Valley Brewing,2,NaN,29486,22425
31269,England,Tom Woods,51,NaN,53,31270
32134,France,La Ferme de Kergador (Louis Pierre Le Meur),1,NaN,27980,32135
25212,Netherlands,Hettingabier,10,NaN,10224,25213
30572,England,Hop Studio,53,NaN,14557,30573
19354,Italy,Terra del Sole,1,NaN,13961,19355
9339,Canada,Axe & Barrel Brewing Company,6,43343,NaN,9340
17391,Ukraine,Ohtyrskyj Pyvovarnyj Zavod (Obolon),10,NaN,17472,17392
15726,"United States, Indiana",Mad Anthony's Old State Alehouse,0,21785,NaN,15727
9694,Bulgaria,Ale House,1,21746,NaN,9695


In [38]:
print(len(full_breweries))
print(len(breweries_matched),len(breweries_advocate),len(breweries_ratebeer))
print(-len(breweries_matched)+len(breweries_advocate)+len(breweries_ratebeer))

32666
8281 16758 24189
32666


This confirms the number of single breweries we computed in data_understanding.ipynb.

In [39]:
full_breweries.to_csv(os.path.join(FULL_PATH,'breweries.csv'),index=False)

In [40]:
test_breweri = pd.read_csv(os.path.join(FULL_PATH,'breweries.csv'))
print(len(test_breweri))
del test_breweri

32666


**Users dataset**

We prefer to consider the users approx rather than the users of the matched dataset. We saw that the users do have duplicates (unlike the normal file). We decided to take the different users in the approx file as a single user even for a lower similitude value (no sim value is below 0.8006407690254358). We consider the effect of this to be negligeable as it is a small percent of data. (like in the brewery file).

In [186]:
users_matched = pd.read_csv(os.path.join(MATCHED_PATH,'users_approx.csv'),header=1)
users_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'users.csv'))
users_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'users.csv'))

ratings_matched = pd.read_csv(os.path.join(MATCHED_PATH,'ratings.csv'),header=1)#needed 


print("Length of the three datasets:\n-advocate:",len(users_advocate),"\n-matched:", len(users_matched),"\n-ratebeer:", len(users_ratebeer))

#Already delete the breweries presents in matched
users_ratebeer_solo = users_ratebeer[~users_ratebeer.user_name.isin(users_matched['user_name.1'])]
users_advocate_solo = users_advocate[~users_advocate.user_name.isin(users_matched['user_name'])]

print("New length of:\n-advocate:",len(users_advocate_solo),"\n-ratebeer:", len(users_ratebeer_solo))

Length of the three datasets:
-advocate: 153704 
-matched: 3341 
-ratebeer: 70174
New length of:
-advocate: 150388 
-ratebeer: 66833


In [187]:
users_matched.head(2)

,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined.1,location.1,nbr_ratings.1,user_id.1,user_name.1,user_name_lower.1,sim
0,1.483009e+09,Spain,3,0,magicuenca.1185749,MAGICuenca,magicuenca,1.484046e+09,Spain,89,442761,MAGICuenca91,magicuenca91,0.904534
1,1.220868e+09,Germany,6,6,erzengel.248045,Erzengel,erzengel,1.224324e+09,Germany,8781,83106,Erzengel,erzengel,1.000000


In [188]:
users_duplicates_ratebeer = users_matched[users_matched['user_name.1'].duplicated(keep=False)]
users_duplicates_advocate = users_matched[users_matched['user_name'].duplicated(keep=False)]
users__not_duplicated_advocate = users_matched[~users_matched['user_name'].duplicated(keep=False)]#For later

print("Number of duplicated/tripled Ratbeer users",len(users_duplicates_ratebeer))
print("Number of duplicated/tripled Advocate users",len(users_duplicates_advocate))

Number of duplicated/tripled Ratbeer users 0
Number of duplicated/tripled Advocate users 47


Both advocate and ratebeer dataset have as columns ['user_id', 'location', 'user_name','user_name_lower', 'joined','nbr_ratings']. Advocate has nbr_reviews in addition. Matched has those columns with .1 referring to ratebeer. It also has a column 'sim' that we will drop. The end format we want to have: [general_id, old_user_id_advocate, old_user__id_ratebeer, location, user_name_lower, joined_advocate, joined_ratebeer and nbr_ratings]. We think that the other columns will not be usefull for our analysis. The next cell shows that the user_name_lower are equal between the two datasets. For the location we will in opposition to the brewery give priority to the advocate dataset, as it is this time the one having one correspondence to multiple ratebeer users.

In [189]:
users_duplicates_advocate.sample(2)

,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined.1,location.1,nbr_ratings.1,user_id.1,user_name.1,user_name_lower.1,sim
2350,1.334225e+09,Norway,7,0,morten.672301,Morten,morten,1.317982e+09,Norway,1,137302,mortenag,mortenag,0.845154
1271,1.218103e+09,Netherlands,38,35,schuim.240905,schuim,schuim,1.007118e+09,Netherlands,34,2462,schuim,schuim,1.000000


In [190]:
users_duplicates_advocate[users_duplicates_advocate['user_id']=='lonestar.677281'].head(2)

,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined.1,location.1,nbr_ratings.1,user_id.1,user_name.1,user_name_lower.1,sim
562,1.337422e+09,"United States, Texas",1,1,lonestar.677281,Lonestar,lonestar,1.081332e+09,"United States, Texas",13,11446,oneStar,onestar,0.822609
2059,1.337422e+09,"United States, Texas",1,1,lonestar.677281,Lonestar,lonestar,1.162984e+09,"United States, Texas",6,44744,LONESTAR,lonestar,1.000000


In [191]:
#We are going to build a new dataset instead of playing with the dataset that contains every information for duplicates.

unique_advocate_user_user_name = users_duplicates_advocate['user_name'].unique()
new_matched_user_duplicate = pd.DataFrame(columns=['location', 'joined_advocate', 'old_user_id_advocate','joined_ratebeer','old_user_id_ratebeer','user_name','nbr_ratings_total'])

for user_name in unique_advocate_user_user_name:  # Replace with your actual loop condition
    # Define data for each row
    location = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['location'].iloc[0]#Always take the first as it is the same for the two/three of them
    joined_advocate = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['joined'].iloc[0]
    joined_ratebeer = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['joined.1'].min() #take min value of joined time (we want the first time they joined)

    old_user_id_advocate = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['user_id'].iloc[0]
    old_user_id_ratebeer = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['user_id.1']
    name = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['user_name'].iloc[0]

    list_old_ratebeer_id = []
    for id_ratbeer in (old_user_id_ratebeer):
        list_old_ratebeer_id.append(id_ratbeer)

    total_ratings_advocate = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['nbr_ratings'].iloc[0]
    total_ratings_ratebeer = users_duplicates_advocate[users_duplicates_advocate['user_name']==user_name]['nbr_ratings.1'].sum()

    nbr_ratings_total = total_ratings_advocate+ total_ratings_ratebeer

    # Append the new row to the DataFrame
    new_row = pd.DataFrame({'location': location, 'user_name': name,'old_user_id_ratebeer':[list_old_ratebeer_id],'old_user_id_advocate':old_user_id_advocate,'joined_advocate':joined_advocate,'joined_ratebeer':joined_ratebeer,'nbr_ratings_total':nbr_ratings_total})
    new_matched_user_duplicate = pd.concat([new_matched_user_duplicate, new_row], ignore_index=True)

new_matched_user_duplicate.sample(10)


C:\Users\valen\AppData\Local\Temp\ipykernel_10596\314378586.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_matched_user_duplicate = pd.concat([new_matched_user_duplicate, new_row], ignore_index=True)


,location,joined_advocate,old_user_id_advocate,joined_ratebeer,old_user_id_ratebeer,user_name,nbr_ratings_total
8,"United States, Massachusetts",1.336298e+09,maximusmaximus.675527,1.108897e+09,"[20099, 356497]",Maximusmaximus,112
1,England,1.245751e+09,leighton.343447,1.209204e+09,"[74136, 257478]",leighton,19601
10,Australia,1.392462e+09,azza.782374,1.174903e+09,"[343397, 355085, 51942]",Azza,56
21,"United States, Ohio",1.104318e+09,beerbeerbeerbeer.12454,1.107256e+09,"[19505, 411715]",beerbeerbeerbeer,8
16,Norway,1.334225e+09,morten.672301,1.140606e+09,"[137013, 33840, 137302]",Morten,16
11,"United States, Indiana",1.399975e+09,indianabeerman.799605,1.070104e+09,"[9581, 283897]",indianabeerman,668
7,Canada,1.156068e+09,nighthawk.93517,1.150452e+09,"[124623, 38708]",nighthawk,212
5,"United States, California",1.295521e+09,chadski.555343,1.381054e+09,"[282710, 397860]",chadski,406
15,Brazil,1.365502e+09,marcelo.727734,1.146823e+09,"[294202, 36866]",Marcelo,11
4,"United States, Texas",1.337422e+09,lonestar.677281,1.081332e+09,"[11446, 44744]",Lonestar,20


In [192]:
users_matched_not_duplicated = users__not_duplicated_advocate.drop(columns=['nbr_reviews','user_name.1','user_name_lower','location','user_name_lower.1','sim'])
users_matched_not_duplicated = users_matched_not_duplicated.rename(columns={'joined': 'joined_advocate','joined.1': 'joined_ratebeer','user_id': 'old_user_id_advocate','user_id.1': 'old_user_id_ratebeer','location.1':'location'})
users_matched_not_duplicated["nbr_ratings_total"] = users_matched_not_duplicated.nbr_ratings+users_matched_not_duplicated["nbr_ratings.1"]
users_matched_not_duplicated = users_matched_not_duplicated.drop(columns=['nbr_ratings','nbr_ratings.1'])
users_matched_not_duplicated.sample(5)

,joined_advocate,old_user_id_advocate,user_name,joined_ratebeer,location,old_user_id_ratebeer,nbr_ratings_total
2886,1.190714e+09,ender796.160938,ender796,1.378375e+09,"United States, Ohio",278093,3
3185,1.275041e+09,tlyerly.463853,tlyerly,1.388920e+09,"United States, South Carolina",295990,7
179,1.289560e+09,xmnwildx12.524792,XmnwildX12,1.295521e+09,"United States, Minnesota",121664,6517
1550,1.406628e+09,brezina.835522,brezina,1.341310e+09,Czech Republic,200029,7
174,1.223374e+09,wakeandbake.255846,WakeandBake,1.214647e+09,"United States, Michigan",77562,984


In [193]:
#Concat the two datasets together
new_matched_user = pd.concat([users_matched_not_duplicated, new_matched_user_duplicate], ignore_index=True)
new_matched_user.sample(3)

,joined_advocate,old_user_id_advocate,user_name,joined_ratebeer,location,old_user_id_ratebeer,nbr_ratings_total
987,1.234091e+09,flandersned.296268,FlandersNed,1.233918e+09,"United States, Indiana",86986,509
780,1.397815e+09,paologalloni.795270,paologalloni,1.381831e+09,Italy,283955,166
2047,1.274868e+09,itfitz.463312,itfitz,1.274868e+09,"United States, California",107075,15


In [194]:
users_matched_not_duplicated_good_amount_ratings = new_matched_user.copy()
for id in ratings_matched['user_id']:
    users_matched_not_duplicated_good_amount_ratings.loc[users_matched_not_duplicated_good_amount_ratings['old_user_id_advocate'] == id, 'nbr_ratings_total'] -= 1

print(new_matched_user.nbr_ratings_total.sum())
print(users_matched_not_duplicated_good_amount_ratings.nbr_ratings_total.sum())

1608147
1586183


In [195]:
users_ratebeer_solo = users_ratebeer_solo.rename(columns={'user_id': 'old_user_id_ratebeer','joined':'joined_ratebeer','nbr_ratings':'nbr_ratings_total'})
users_ratebeer_solo['old_user_id_advocate'] = np.nan
users_ratebeer_solo['joined_advocate'] = np.nan
users_ratebeer_solo.sample(3)  

,nbr_ratings_total,old_user_id_ratebeer,user_name,joined_ratebeer,location,old_user_id_advocate,joined_advocate
39686,2,156497,crousejw12,1.328699e+09,"United States, Alabama",NaN,NaN
39473,10,137650,seraphimj,1.318414e+09,"United States, Minnesota",NaN,NaN
67268,3,397737,Vivev,1.452942e+09,NaN,NaN,NaN


In [196]:
users_advocate_solo = users_advocate_solo.rename(columns={'user_id': 'old_user_id_advocate','joined':'joined_advocate','nbr_ratings':'nbr_ratings_total'})
users_advocate_solo['old_user_id_ratebeer'] = np.nan
users_advocate_solo['joined_ratebeer'] = np.nan
users_advocate_solo.sample(3)  

,nbr_ratings_total,nbr_reviews,old_user_id_advocate,user_name,joined_advocate,location,old_user_id_ratebeer,joined_ratebeer
143206,1,0,dan708.755645,dan708,1.380103e+09,"United States, Michigan",NaN,NaN
125619,1,0,wannick3.995230,Wannick3,1.433326e+09,NaN,NaN,NaN
122542,1,0,marsupermammal.1042077,Marsupermammal,1.442138e+09,"United States, Pennsylvania",NaN,NaN


In [197]:
full_users = pd.concat([users_matched_not_duplicated_good_amount_ratings,users_ratebeer_solo, users_advocate_solo], ignore_index=True)
full_users['id'] = range(1, len(full_users) + 1)
full_users.sample(2)


,joined_advocate,old_user_id_advocate,user_name,joined_ratebeer,location,old_user_id_ratebeer,nbr_ratings_total,nbr_reviews,id
178393,1.389179e+09,drboud.774884,drboud,NaN,"United States, North Carolina",NaN,6,0.0,178394
163707,1.393672e+09,gerry05060.785940,Gerry05060,NaN,"United States, Vermont",NaN,3,0.0,163708


In [243]:
print(len(full_users))
print(len(users_matched),len(users_advocate),len(users_ratebeer))
print(len(users_matched_not_duplicated_good_amount_ratings)+len(users_advocate_solo)+len(users_ratebeer_solo))

220537
3341 153704 70174
220537


In [199]:
test = full_users[full_users['user_name'].duplicated(keep=False)]
test[test['user_name']=='Elwood']

,joined_advocate,old_user_id_advocate,user_name,joined_ratebeer,location,old_user_id_ratebeer,nbr_ratings_total,nbr_reviews,id
990,1.152871e+09,elwood.88673,Elwood,1.374314e+09,Canada,270235,6,NaN,991
4952,NaN,NaN,Elwood,1.235041e+09,"United States, Virginia",87609,2966,NaN,4953


Oh so in different dataset people can have the same user_name. We need to be carefull avout this. If we treat the data we need to use id.

In [200]:
full_users.to_csv(os.path.join(FULL_PATH,'users.csv'),index=False)

In [201]:
test_user = pd.read_csv(os.path.join(FULL_PATH,'users.csv'))
print(len(test_user))
del test_user

220537


C:\Users\valen\AppData\Local\Temp\ipykernel_10596\1172294355.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  test_user = pd.read_csv(os.path.join(FULL_PATH,'users.csv'))


**Beer dataset**

In this dataset we saw that there were no duplicates. We do not need to make the carefull analysis we made before. However we still need to look at the matched dataset. We also need to link the new brewery id, but we also need to count the number of ratings.

In [202]:
beers_matched = pd.read_csv(os.path.join(MATCHED_PATH,'beers.csv'),header=1)
beers_advocate = pd.read_csv(os.path.join(ADVOCATE_PATH,'beers.csv'))
beers_ratebeer = pd.read_csv(os.path.join(RATEBEER_PATH,'beers.csv'))

ratings_matched = pd.read_csv(os.path.join(MATCHED_PATH,'ratings.csv'),header=1)#needed 
full_breweries_for_beers = pd.read_csv(os.path.join(FULL_PATH,'breweries.csv'))#needed



print("Length of the three datasets:\n-advocate:",len(beers_advocate),"\n-matched:", len(beers_matched),"\n-ratebeer:", len(beers_ratebeer))

#Already delete the breweries presents in matched
beers_ratebeer_solo = beers_ratebeer[~beers_ratebeer.beer_id.isin(beers_matched['beer_id.1'])]
beers_advocate_solo = beers_advocate[~beers_advocate.beer_id.isin(beers_matched['beer_id'])]

print("New length of:\n-advocate:",len(beers_advocate_solo),"\n-ratebeer:", len(beers_ratebeer_solo))

Length of the three datasets:
-advocate: 280823 
-matched: 45640 
-ratebeer: 442081
New length of:
-advocate: 235183 
-ratebeer: 396441


In [203]:
beers_matched.sample(2)

,abv,avg,avg_computed,avg_matched_valid_ratings,ba_score,beer_id,beer_name,beer_wout_brewery_name,brewery_id,brewery_name,...,brewery_id.1,brewery_name.1,nbr_matched_valid_ratings.1,nbr_ratings.1,overall_score,style.1,style_score,zscore.1,diff,sim
36584,11.7,3.48,3.677955,3.470385,80.0,88047,Death,Death,22157,Rivertown Brewing Co.,...,11245,Rivertown Brewery,48,48,60.0,Imperial Stout,10.0,-0.198514,1.0,1.0
29005,5.2,3.63,3.675000,3.600000,NaN,274565,Pauline,Pauline,48752,Two Plumbers Brewery + Arcade,...,30677,Two Plumbers Brewery + Arcade,2,2,NaN,Golden Ale/Blond Ale,NaN,-0.509111,1.0,1.0


In [204]:
beers_new = beers_matched.drop(columns=['avg','beer_wout_brewery_name.1','avg.1','avg_matched_valid_ratings.1','nbr_reviews','beer_name.1','brewery_name.1','brewery_name','avg_computed','avg_computed.1','avg_matched_valid_ratings','ba_score','beer_wout_brewery_name','sim','diff','zscore','zscore.1','overall_score','style_score','nbr_matched_valid_ratings','nbr_matched_valid_ratings.1'])

In [205]:
beers_new.sample(2)

,abv,beer_id,beer_name,brewery_id,bros_score,nbr_ratings,style,abv.1,beer_id.1,brewery_id.1,nbr_ratings.1,style.1
20951,4.5,141742,Spellbound Session Porter,34780,NaN,11,American Porter,4.5,292681,21087,8,Porter
34248,6.6,235589,Hoity Toity,33255,NaN,0,American Strong Ale,6.6,292421,18745,1,American Strong Ale


In [206]:
print((beers_new['abv'] == beers_new['abv.1']).all())

True


Can use just one

In [208]:
#We are going to build a new dataset instead of playing with the dataset that contains every information for duplicates.
#Not sure about valid ratings so prefer to keep it as it is for now
new_matched_beer = pd.DataFrame(columns=['abv', 'old_beer_id_advocate', 'old_beer_id_ratebeer','beer_name','brewery_id','bros_score','nbr_ratings','style_advocate','style_ratebeer'])

for index, row in beers_new.iterrows():
    # Define data for each row
    abv = row['abv']
    beer_id_advocate = row['beer_id']
    beer_id_ratebeer = row['beer_id.1']
    beer_name = row['beer_name']
    bros_score = row['bros_score']
    style_advocate = row['style']
    style_ratebeer = row['style.1']


    brewery_id = full_breweries_for_beers[full_breweries_for_beers['old_ratebeer_id']==row['brewery_id.1']]['id']


    total_ratings_advocate = row['nbr_ratings']
    total_ratings_ratebeer = row['nbr_ratings.1']

    nbr_ratings_total = total_ratings_advocate+ total_ratings_ratebeer

    # Append the new row to the DataFrame
    new_row = pd.DataFrame({'abv':abv, 'old_beer_id_advocate':beer_id_advocate, 'old_beer_id_ratebeer':beer_id_ratebeer,'beer_name':beer_name,'brewery_id':brewery_id,'bros_score':bros_score,'nbr_ratings':nbr_ratings_total,'style_advocate':style_advocate,'style_ratebeer':style_ratebeer})
    new_matched_beer = pd.concat([new_matched_beer, new_row], ignore_index=True)

new_matched_beer.sample(10)

C:\Users\valen\AppData\Local\Temp\ipykernel_10596\1440708053.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_matched_beer = pd.concat([new_matched_beer, new_row], ignore_index=True)


,abv,old_beer_id_advocate,old_beer_id_ratebeer,beer_name,brewery_id,bros_score,nbr_ratings,style_advocate,style_ratebeer
29269,10.5,257321,466109,Middle Earth,5660,NaN,3,American Double / Imperial Stout,Imperial Stout
9228,6.6,251223,419598,Cerveza Negra,2416,NaN,8,Euro Dark Lager,Dunkel/Tmavý
33181,5.5,139671,284998,Strawberry Whale Cake,6128,NaN,88,Cream Ale,Fruit Beer
41233,6.0,257110,477972,Cold Town,7100,NaN,2,Cream Ale,Cream Ale
1230,4.2,103759,49634,Kilmington Best,278,NaN,10,English Bitter,Bitter
30330,7.1,135044,279375,Peloton Saison,5789,NaN,11,Saison / Farmhouse Ale,Saison
11064,5.0,15304,30183,Godelief,3194,NaN,29,Belgian Pale Ale,Belgian Ale
6944,13.5,58896,107790,Xyauyù Etichetta Argento (Silver),1477,NaN,127,English Barleywine,Barley Wine
45312,3.5,30385,62658,Mussel Ridge Summer Ale,8047,NaN,2,American Blonde Ale,Golden Ale/Blond Ale
37796,3.0,231373,421988,Cucumber And Lime Maigre,6659,NaN,28,Fruit / Vegetable Beer,Berliner Weisse


In [214]:
new_matched_beer_good_amount_ratings = new_matched_beer.copy()
for id in ratings_matched['beer_id']:
    new_matched_beer_good_amount_ratings.loc[new_matched_beer_good_amount_ratings['old_beer_id_advocate'] == id, 'nbr_ratings'] -= 1

print(new_matched_beer.nbr_ratings.sum())
print(new_matched_beer_good_amount_ratings.nbr_ratings.sum())

1976606
1954642


In [218]:
beers_ratebeer_solo_new = beers_ratebeer_solo.copy()
beers_ratebeer_solo_new = beers_ratebeer_solo_new.drop(columns=['brewery_name','overall_score','style_score','avg','avg_computed','zscore','nbr_matched_valid_ratings','avg_matched_valid_ratings'])
beers_ratebeer_solo_new = beers_ratebeer_solo_new.rename(columns={'beer_id': 'old_beer_id_ratebeer','style':'style_ratebeer'})

beers_ratebeer_solo_new = pd.merge(beers_ratebeer_solo_new,full_breweries_for_beers[['old_ratebeer_id', 'id']],how='left', left_on='brewery_id',right_on='old_ratebeer_id')
beers_ratebeer_solo_new = beers_ratebeer_solo_new.drop(columns=['brewery_id','old_ratebeer_id'])
beers_ratebeer_solo_new = beers_ratebeer_solo_new.rename(columns={'id': 'brewery_id'})

beers_ratebeer_solo_new['old_beer_id_advocate'] = np.nan
beers_ratebeer_solo_new['bros_score'] = np.nan
beers_ratebeer_solo_new['style_advocate']=np.nan

beers_ratebeer_solo_new.sample(3)

,old_beer_id_ratebeer,beer_name,style_ratebeer,nbr_ratings,abv,brewery_id,old_beer_id_advocate,bros_score,style_advocate
73145,187059,Steinfels Dunkel,Dunkelweizen,2,NaN,2199,NaN,NaN,NaN
160312,351104,Kuracali Session Brown,Brown Ale,1,4.5,3939,NaN,NaN,NaN
172236,95407,Schooners Blueberry Wheat,Fruit Beer,2,NaN,27149,NaN,NaN,NaN


In [221]:
beers_advocate_solo.sample(1)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
190306,198703,Pound Of Flesh,35374,Haw River Farmhouse Ales,American Wild Ale,2,2,4.15,NaN,NaN,NaN,4.155,NaN,0,NaN


In [240]:
beers_advocate_solo_new = beers_advocate_solo.copy()
beers_advocate_solo_new = beers_advocate_solo_new.drop(columns=['brewery_name','nbr_reviews','ba_score','avg','avg_computed','zscore','nbr_matched_valid_ratings','avg_matched_valid_ratings'])
beers_advocate_solo_new = beers_advocate_solo_new.rename(columns={'beer_id': 'old_beer_id_advocate','style':'style_advocate'})


full_breweries_for_beers_exploded = full_breweries_for_beers.explode('old_advocate_id')

beers_advocate_solo_new['brewery_id'] = beers_advocate_solo_new['brewery_id'].astype(str)
full_breweries_for_beers_exploded['old_advocate_id'] = full_breweries_for_beers_exploded['old_advocate_id'].astype(str)
full_breweries_for_beers_exploded['id'] = full_breweries_for_beers_exploded['id'].astype(str)

beers_advocate_solo_new = pd.merge(beers_advocate_solo_new,full_breweries_for_beers_exploded[['old_advocate_id', 'id']],how='left', left_on='brewery_id',right_on='old_advocate_id')
beers_advocate_solo_new = beers_advocate_solo_new.drop(columns=['brewery_id','old_advocate_id'])
beers_advocate_solo_new = beers_advocate_solo_new.rename(columns={'id': 'brewery_id'})

beers_advocate_solo_new['old_beer_id_ratebeer'] = np.nan
beers_advocate_solo_new['style_ratebeer']=np.nan

beers_advocate_solo_new.head()


,old_beer_id_advocate,beer_name,style_advocate,nbr_ratings,bros_score,abv,brewery_id,old_beer_id_ratebeer,style_ratebeer
0,166064,Nashe Moskovskoe,Euro Pale Lager,0,NaN,4.7,8236,NaN,NaN
1,166065,Nashe Pivovskoe,Euro Pale Lager,0,NaN,3.8,8236,NaN,NaN
2,166066,Nashe Shakhterskoe,Euro Pale Lager,0,NaN,4.8,8236,NaN,NaN
3,166067,Nashe Zhigulevskoe,Euro Pale Lager,0,NaN,4.0,8236,NaN,NaN
4,166063,Zhivoe,Euro Pale Lager,0,NaN,4.5,8236,NaN,NaN


In [241]:
full_beers = pd.concat([new_matched_beer_good_amount_ratings,beers_ratebeer_solo_new, beers_advocate_solo_new], ignore_index=True)
full_beers['id'] = range(1, len(full_beers) + 1)
full_beers.sample(2)


,abv,old_beer_id_advocate,old_beer_id_ratebeer,beer_name,brewery_id,bros_score,nbr_ratings,style_advocate,style_ratebeer,id
584745,4.7,197818,NaN,Belle Isle Blonde,13936,NaN,4,American Blonde Ale,NaN,584746
82481,7.2,NaN,197333,Buonconvento Ambrio,18807,NaN,0,NaN,Abbey Dubbel,82482


In [242]:
print(len(full_beers))
print(len(beers_matched),len(beers_advocate),len(beers_ratebeer))
print(len(new_matched_beer_good_amount_ratings)+len(beers_advocate_solo_new)+len(beers_ratebeer_solo_new))

677264
45640 280823 442081
677264


In [244]:
full_beers.to_csv(os.path.join(FULL_PATH,'beers.csv'),index=False)

In [245]:
test_beer = pd.read_csv(os.path.join(FULL_PATH,'beers.csv'))
print(len(test_beer))
del test_beer

C:\Users\valen\AppData\Local\Temp\ipykernel_10596\2466902068.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  test_beer = pd.read_csv(os.path.join(FULL_PATH,'beers.csv'))


677264
